In [1]:
from typing import Any

from typing_extensions import Self

from pydantic import BaseModel, ValidationError, model_validator


class UserModel(BaseModel, validate_assignment=True):
    username: str
    password1: str
    password2: str

    @model_validator(mode='before')
    @classmethod
    def check_card_number_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            assert (
                'card_number' not in data
            ), 'card_number should not be included'
        return data

    @model_validator(mode='after')
    def check_passwords_match(self) -> Self:
        pw1 = self.password1
        pw2 = self.password2
        if pw1 is not None and pw2 is not None and pw1 != pw2:
            raise ValueError('passwords do not match')
        return self

In [2]:
test = UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn')
print(test.model_dump())
test.password2 = 'dafdaf'
print(test.model_dump())

{'username': 'scolvin', 'password1': 'zxcvbn', 'password2': 'zxcvbn'}


ValidationError: 1 validation error for UserModel
  Value error, passwords do not match [type=value_error, input_value=UserModel(username='scolv...bn', password2='dafdaf'), input_type=UserModel]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error

In [1]:
import numpy as np
from pydantic import Field, BaseModel, model_validator, computed_field
from warnings import warn
from typing import Any
from typing_extensions import Self

class MultivariateGaussian(BaseModel,
                           validate_assignment=True,
                           arbitrary_types_allowed=True):
    """
    Class to describe a multivariate Gaussian distribution
    """
    mean: np.ndarray = Field(default=np.array([0]),
                             description='Mean of the multivariate Gaussian distribution')
    covariance: np.ndarray = Field(default=np.array([[1]]),
                                   description='Covariance of the multivariate Gaussian distribution')
    
    @computed_field
    @property
    def num_dimensions(self) -> int:
        return len(self.mean)
    

    @model_validator(mode='before')
    @classmethod
    def fields_shape(cls, data: Any) -> Any:
        if isinstance(data, dict):
            mean_shape = data['mean'].shape
            cov_shape = data['covariance'].shape
            if not mean_shape:
                raise ValueError('Mean must be Sized and have a non-empty shape!')
            if len(mean_shape) > 2 or (len(mean_shape) == 2 and 1 not in mean_shape):
                raise ValueError('Mean must be a 1D vector, but array provided has shape ' + str(mean_shape) + '!')
            elif len(mean_shape) == 2:
                msg = 'Mean passed has shape (%d, %d), but it will be flattened to (%d,)' % (mean_shape + (max(mean_shape),))
                warn(msg)
                data['mean'] = data['mean'].flatten()
            if len(cov_shape) != 2:
                raise ValueError('Covariance must be a 2D matrix, but shape provided was ' + str(cov_shape) + '!')
        return data
    
    @model_validator(mode='after')
    def fields_dim(self) -> Self:
        dim = self.num_dimensions
        print('here, dim = ', dim)
        return self


In [3]:
# test = MultivariateGaussian(mean=np.array([[[1,2], [3,4]],[[5,6], [7,8]],[[9,10], [11,12]]]),
#                             cov=0)
# test = MultivariateGaussian(mean=np.eye(2), cov=0)
test = MultivariateGaussian(mean=np.array([[1,2,3]]), covariance=np.eye(2))
print(test.mean)
test.mean = np.array([[1,2,3,4]])
print(test.mean)

here, dim =  3
[1 2 3]
here, dim =  1
[[1 2 3 4]]


/var/folders/89/mkngtyzs54l121mx93tt09z00000gp/T/ipykernel_68657/3841197278.py:36: UserWarning: Mean passed has shape (1, 3), but it will be flattened to (3,)
  warn(msg)
/var/folders/89/mkngtyzs54l121mx93tt09z00000gp/T/ipykernel_68657/3841197278.py:36: UserWarning: Mean passed has shape (1, 4), but it will be flattened to (4,)
  warn(msg)


In [1]:
import numpy as np
from pydantic import Field, BaseModel, model_validator, field_validator, ValidationInfo, computed_field
from warnings import warn
from typing_extensions import Self

class sth(BaseModel, arbitrary_types_allowed=True, validate_assignment=True):
    mean: np.ndarray
    cov: np.ndarray

    # @computed_field
    @property
    def num_dimensions(self) -> int:
        return len(self.mean)
    
    @field_validator('mean', mode='before')
    @classmethod
    def mean_1d(cls, mu: np.ndarray):
        mean_shape = mu.shape
        if not mean_shape:
            raise ValueError('Mean must be Sized and have a non-empty shape!')
        if len(mean_shape) > 2 or (len(mean_shape) == 2 and 1 not in mean_shape):
            raise ValueError('Mean must be a 1D vector, but array provided has shape ' + str(mean_shape) + '!')
        elif len(mean_shape) == 2:
            msg = 'Mean passed has shape (%d, %d), but it will be flattened to (%d,)' % (mean_shape + (max(mean_shape),))
            warn(msg)
            return mu.flatten()
        return mu
    
    @field_validator('mean', mode='after')
    @classmethod
    def mean_dim(cls, mu: np.ndarray, info=ValidationInfo):
        print(info)
        present_mean = getattr(cls, 'mean', None)
        dim = None if present_mean is None else len(present_mean)
        print('here, dim = ', dim)
        return mu


In [2]:
# test = sth(mean=np.array([[[1,2], [3,4]],[[5,6], [7,8]],[[9,10], [11,12]]]),
#                             cov=np.array(0))
# test = sth(mean=np.eye(2), cov=np.array(0))
# test = sth(mean=np.array([[1,2,3]]), cov=np.array(0))
test = sth(mean=np.array([1,2,3]), cov=np.array(0))
print(test.mean)
test.mean = np.array([[1,2,3,4]])
print(test.mean)

ValidationInfo(config={'title': 'sth'}, context=None, data={}, field_name='mean')
here, dim =  None
[1 2 3]
ValidationInfo(config={'title': 'sth'}, context=None, data={'cov': array(0)}, field_name='mean')
here, dim =  None
[1 2 3 4]


/var/folders/89/mkngtyzs54l121mx93tt09z00000gp/T/ipykernel_67910/3324510748.py:25: UserWarning: Mean passed has shape (1, 4), but it will be flattened to (4,)
  warn(msg)


In [1]:
import numpy as np
from pydantic import Field, BaseModel, model_validator, field_validator, ValidationInfo, computed_field
from warnings import warn
from typing import Any
from typing_extensions import Self

class AllChecked(BaseModel, arbitrary_types_allowed=True, validate_assignment=True):
    mean: np.ndarray
    cov: np.ndarray

    @computed_field
    @property
    def num_dimensions(self) -> int:
        return len(self.mean)
    
    @field_validator('mean', mode='before')
    @classmethod
    def mean_1d(cls, mu: np.ndarray)-> np.ndarray:
        mean_shape = mu.shape
        if not mean_shape:
            raise ValueError('Mean must be Sized and have a non-empty shape!')
        if len(mean_shape) > 2 or (len(mean_shape) == 2 and 1 not in mean_shape):
            raise ValueError('Mean must be a 1D vector, but array provided has shape ' + str(mean_shape) + '!')
        elif len(mean_shape) == 2:
            msg = 'Provided mean has shape (%d, %d), but it will be flattened to (%d,)' % \
                (mean_shape + (max(mean_shape),))
            warn(msg)
        return mu.flatten()
    
    @field_validator('cov', mode='before')
    @classmethod
    def cov_2d(cls, sigma: np.ndarray) -> np.ndarray:
        cov_shape = sigma.shape
        if len(cov_shape) != 2:
            raise ValueError('Covariance must be a 2D matrix, but shape provided was ' + str(cov_shape) + '!')
        return sigma
    
    @model_validator(mode='after')
    def fields_dim(self) -> Self:
        dim = self.num_dimensions
        if self.cov.shape != (dim, dim):
            msg = 'Wrong dimensions! Mean has shape ' + str(self.mean.shape)
            msg += ', but covariance has shape ' + str(self.cov.shape)
            raise ValueError(msg)
        return self


    

In [2]:
mu0 = np.array([[1,2,3]]) # np.eye(2)
sig0 = np.eye(3) # np.arange(8).reshape((2,2,2)) # 
# print(type(sig0))
test = AllChecked(mean=mu0, cov=sig0)
print(test.mean)
test.mean = np.array([1,2,3,4])
print(test.mean)

[1 2 3]


/var/folders/89/mkngtyzs54l121mx93tt09z00000gp/T/ipykernel_76442/3556383232.py:26: UserWarning: Mean passed has shape (1, 3), but it will be flattened to (3,)
  warn(msg)


ValidationError: 1 validation error for AllChecked
  Value error, Wrong dimensions! Mean has shape (4,), but covariance has shape (3, 3) [type=value_error, input_value=AllChecked(mean=array([1,...1.]]), num_dimensions=4), input_type=AllChecked]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error